In [35]:
import kiutils.symbol
import os
import glob
import pandas as pd
import numpy as np


"""
loop through Kicad symbol libraries
For each Kicad symbol, check the parts sheet for a matching BR number
    If the BR number exists, check for changes
        If changed, make edits
        Otherwise, don't touch that part
    If the BR number doesn't exist, raise an error. All BR numbers should originate in the parts sheet
    If there is no BR ID field for a part, generate a new BR ID and fill out sheet accordingly using the Kicad symbol fields
            "ID":BR_ID, "Name":symbol.libId, "Description":properties["Description"], "Value":properties["Value"], "Symbol":symbol_path, "Footprint":properties["Footprint"],  "Datasheet":properties["Datasheet"], "Manufacturer":manufacturer, "MPN":mpn, "Category":category
    At the end, create a new row in the vendor sheet

Functions to add:
check_for_duplicate_BRID()
check_for_changes()
update_part_in_sheet()
add_BRID_to_kicad_part()
add_part_to_vendor_sheet()
            
"""
# Not really necessary, but initializes the dataframes. This is at good reference for the columns, at least
parts_df = pd.DataFrame(columns=['BR ID','Name','Description','Value','Symbol','Footprint','Datasheet','Manufacturer','MPN', 'Category'])
vendors_df = pd.DataFrame(columns=['BR ID','Supplier','SPN','Stock'])

ID_list = []
parts_list = []
vendors_list = []

# Reading symbol libraries from our BR symbols folder
# SYMBOLS_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-kicad-lib/Symbols"
# JLC_PATH = r"C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/jlc-scraper/csv/Parts Inventory on JLCPCB.xlsx"
# PARTS_SHEET_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/Kicad/Parts_Library.xlsx"
SYMBOLS_PATH = r"C:/Users/JacobBrotmanKrass/Documents/Test Library/Symbols"
JLC_PATH = r"C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/jlc-scraper/csv/Parts Inventory on JLCPCB.xlsx"
PARTS_SHEET_PATH = "C:/Users/JacobBrotmanKrass/Documents/Test_Parts_Library.xlsx"
VENDORS_SHEET_PATH = "C:/Users/JacobBrotmanKrass/Documents/Test_Vendor_Stock.xlsx"

parts_sheet = pd.read_excel(PARTS_SHEET_PATH, index_col=[0])

parts_df, vendors_df = load_kicad_lib_as_dataframe()

check_for_duplicate_BRIDs(parts_df)
check_for_duplicate_BRIDs(parts_sheet)
detect_new_symbols(parts_df)










BR_Capacitors_0201_
BR_Capacitors_0402_
BR_Capacitors_0603_
BR_Capacitors_0805_
BR_Capacitors_1206_
BR_Capacitors_1210
BR_Connectors
BR_Connectors_FFC
BR_Connectors_JST_GH
BR_Connectors_JST_misc
BR_Connectors_Molex_misc
BR_Connectors_Molex_Picoblade
BR_Crystals
BR_Diodes
BR_Electromechanical
BR_IC
BR_IC_Ethernet
BR_IC_Gate_Drivers
BR_IC_Linear
BR_IC_Motor_Drivers
BR_IC_Regulators
BR_IC_Sensors
BR_Inductors
BR_Modules
BR_Optoelectronic
BR_Resistors_0201
BR_Resistors_0402
BR_Resistors_0603
BR_Resistors_0805
BR_Resistors_1206
BR_Resistors_Shunts
BR_Resistors_Thermistors
BR_Switches
BR_Transistors
No BR ID duplicates found in library/sheet!
No BR ID duplicates found in library/sheet!


In [38]:
new_parts = parts_df[parts_df["BR ID"] == np.nan]
parts_df
## NAN NOT WORKING

,BR ID,Name,Description,Value,Symbol,Footprint,Datasheet,Manufacturer,MPN,Category
0,BRE-000000,C_0201_100nF_25V_X7R_10%,100nF ±10% 25V X7R 0201 Ceramic Capacitor,100nF 25V,BR_Capacitors_0201_:C_0201_100nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R61E104KE14J,Capacitors_0201_
1,BRE-000001,C_0201_10nF_25V_X7R_10%,10nF ±10% 25V X7R 0201 Ceramic Capacitor,10nF 25V,BR_Capacitors_0201_:C_0201_10nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://search.murata.co.jp/Ceramy/image/img/A...,Murata Electronics,GRM033R71E103KE14D,Capacitors_0201_
2,NaN,C_0201_123nF_50V_C0G_1%_1,123nF ±1% 50V C0G 0201 Ceramic Capacitor,123nF 50V 1%,BR_Capacitors_0201_:C_0201_123nF_50V_C0G_1%_1,BR_Passives:C_0201_0603Metric-minimized,https://bluerobotics.com/,Murata Electronics,MNP1111,Capacitors_0201_
3,BRE-000002,C_0201_1nF_50V_C0G_1%,1nF ±1% 50V C0G 0201 Ceramic Capacitor,1nF 50V 1%,BR_Capacitors_0201_:C_0201_1nF_50V_C0G_1%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM0335C1H102JE01D,Capacitors_0201_
4,BRE-000003,C_0201_1uF_16V_X5R_10%,1uF ±10% 16V X5R 0201 Ceramic Capacitor,1uF 16V,BR_Capacitors_0201_:C_0201_1uF_16V_X5R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.mouser.com/datasheet/2/281/1/GRM03...,Murata Electronics,GRM033R61C105ME15D,Capacitors_0201_
...,...,...,...,...,...,...,...,...,...,...
429,BRE-000428,NTGS5120PT1G,MOSFET P-CH 60V 1.8A 6TSOP,NTGS5120PT1G,BR_Transistors:NTGS5120PT1G,BR_SOP:TSOP-6_ON_SEMI,https://www.onsemi.com/pdf/datasheet/ntgs5120p...,onsemi,NTGS5120PT1G,Transistors
430,BRE-000429,NXV55UNR,"N-Channel 30 V 1.9A (Ta) 340mW (Ta), 2.1W (Tc)...",NXV55UNR,BR_Transistors:NXV55UNR,BR_SOT:SOT-23,https://assets.nexperia.com/documents/data-she...,Nexperia USA Inc.,NXV55UNR,Transistors
431,BRE-000430,SI2305CDS-T1-GE3,Mosfet P-ch 8V 5.8A SOT23-3,SI2305CDS-T1-GE3,BR_Transistors:SI2305CDS-T1-GE3,Package_TO_SOT_SMD:SOT-23,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Vishay,SI2305CDS-T1-GE3,Transistors
432,BRE-000431,SQJ974EP,Mosfet Array 100V 30A (Tc) 48W Surface Mount P...,SQJ974EP,BR_Transistors:SQJ974EP,BR_SOP:PowerPAK_SO-8_Dual,https://www.vishay.com/docs/78092/sqj974ep.pdf,Vishay Siliconix,SQJ974EP-T1_GE3,Transistors


In [34]:
def check_for_duplicate_BRIDs(parts_df):

    # This returns a Series of booleans indicating whether a given entry is a duplicate or not
    if 'BR ID' in parts_df.columns:
        dupes = parts_df.duplicated(subset=['BR ID'], keep=False)
    else:
        dupes = parts_df.index.duplicated(keep=False)
    offenders = parts_df[dupes]

    assert len(offenders) == 0,  f"The following Kicad parts have the same BR IDs. One of em aint right. YOU MUST RESOLVE THIS ERROR GODDAMNIT.\n {offenders}"
    print("No BR ID duplicates found in library/sheet!")

def detect_new_symbols(parts_df):
    new_parts = parts_df[parts_df["BR ID"] == np.nan]
    if len(new_parts) > 0:
        print("ahhhh")

def load_kicad_lib_as_dataframe():
    os.chdir(SYMBOLS_PATH)
    for lib_file in glob.glob("*.kicad_sym"):

        # Extract library nickname/category -- e.g., 0402_Capacitors
        lib_nickname = lib_file.replace(".kicad_sym", "")

        # Skip these libraries, they don't need to be documented (obsolete or not actual parts)
        if (lib_nickname == "BR~Deprecated") or (lib_nickname == "BR_Virtual_Parts"):
            continue
        print(lib_nickname)

        # Open symbol library
        lib_path = os.path.join(SYMBOLS_PATH, lib_file)
        symbol_lib = kiutils.symbol.SymbolLib().from_file(lib_path)
        
        # The Category is the library nickname, without the BR_ at the beginning
        category = lib_nickname[3:]



        # For each symbol in a given library, populate a new row in the Parts dataframe
        for symbol in symbol_lib.symbols:

            # Symbol path in Kicad
            symbol_path = f"{lib_nickname}:{symbol.entryName}"

            # Grab all the properties from the Kicad Symbol
            properties = {property.key: property.value for property in symbol.properties}

            # Remove a weird decoding error with the plus/minus sign
            bad_char = b'\xc3\x82'.decode()
            properties["Description"] = properties["Description"].replace(bad_char, '')

            # Some parts don't have a manufacturer and manufacturer part number -- deal with this some other time, for now just populate "None"
            if "Manufacturer" in properties:
                manufacturer = properties["Manufacturer"]
                mpn = properties["Manufacturer Part Num"]
            else:
                manufacturer = "None"
                mpn = "None"

            if "BR ID" in properties:
                BR_ID = properties["BR ID"]
                if len(BR_ID) < 7:
                    BR_ID = np.nan
            else:
                BR_ID = np.nan
            # Append a dictionary of all part properties to the parts list -- this will be converted to a Pandas dataframe at the end
            parts_list.append({"BR ID":BR_ID, "Name":symbol.libId, "Description":properties["Description"], "Value":properties["Value"], "Symbol":symbol_path, "Footprint":properties["Footprint"],  "Datasheet":properties["Datasheet"], "Manufacturer":manufacturer, "MPN":mpn, "Category":category})

            # Extract all supplier-related properties: supplier X with suppler number X
            supplier_properties = {property: properties[property] for property in properties if property[:8]=="Supplier"}
            supplier_numbers = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]=='P'}
            supplier_names = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]!='P'}

            # Ignore anything that looks like this
            null_strings = ["", " ", "-", "--", "~", "NA", "N/A"]

            # Loop through and add vendors and the respective supplier number when the number X at the end matches (supplier 1 --> supplier part num 1)
            for name in supplier_names:
                for number in supplier_numbers:
                    if name[-1] == number[-1]:
                        if supplier_numbers[number] not in null_strings:

                            # Append dictionary of supplier properties for each SPN to the vendors list
                            # this will be converted to Pandas dataframe at the end
                            vendors_list.append({"BR ID":BR_ID, "Supplier":supplier_names[name], "SPN":supplier_numbers[number], "Stock":0})



    # Create Pandas dataframes from these lists of dictionaries
    # Think of each dictionary as a row in the table
    parts_df = pd.DataFrame(parts_list)
    vendors_df = pd.DataFrame(vendors_list)
    
    return parts_df, vendors_df
    

            